In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import SGD,Adam,RMSprop
from keras.utils import to_categorical
from keras import regularizers

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train_real.csv')
test = pd.read_csv('test_real.csv')
sample_sub = pd.read_csv('sample_submission(1).csv')

In [3]:
submission_df = pd.DataFrame()
submission_df['Id'] = test['Id']

In [4]:
train.head()

,Id,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil,label
0,1,594,738,167,48,243,421,759,964,751,114,0,0,1,0,Soil_Type_23,-1
1,2,343,103,212,43,239,25,861,818,480,208,0,0,1,0,Soil_Type_10,-1
2,3,563,233,167,77,228,555,928,869,452,360,1,0,0,0,Soil_Type_1,-1
3,4,715,602,136,193,289,175,830,979,684,164,0,0,1,0,Soil_Type_32,-1
4,5,672,519,136,231,292,122,869,971,625,45,0,0,1,0,Soil_Type_32,-1


In [5]:
sample_sub.head()

,Id,Predictions
0,1,-1
1,2,-1
2,3,-1
3,4,-1
4,5,-1


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12395 entries, 0 to 12394
Data columns (total 17 columns):
Id                   12395 non-null int64
Altitude             12395 non-null int64
Azimuthal_angle      12395 non-null int64
Incline              12395 non-null int64
H_dist_Hydro         12395 non-null int64
V_dist_Hydro         12395 non-null int64
H_dist_Road          12395 non-null int64
Hillshade_9am        12395 non-null int64
Hillshade_Noon       12395 non-null int64
Hillshade_3pm        12395 non-null int64
H_dist_Fire          12395 non-null int64
Wilderness_Area_A    12395 non-null int64
Wilderness_Area_B    12395 non-null int64
Wilderness_Area_C    12395 non-null int64
Wilderness_Area_D    12395 non-null int64
Soil                 12395 non-null object
label                12395 non-null int64
dtypes: int64(16), object(1)
memory usage: 1.6+ MB


In [7]:
#Importing LabelEncoder from Sklearn

from sklearn.preprocessing import LabelEncoder
label_encoder_soil = LabelEncoder()

In [8]:
# Transforming sex column values using label Encoder

train.iloc[:,15]  = label_encoder_soil.fit_transform(train.iloc[:,15])
test.iloc[:,15]  = label_encoder_soil.fit_transform(test.iloc[:,15])

In [9]:
train = train.drop(['Id'],axis=1)
test = test.drop(['Id'],axis=1)

In [10]:
train.head()

,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil,label
0,594,738,167,48,243,421,759,964,751,114,0,0,1,0,9,-1
1,343,103,212,43,239,25,861,818,480,208,0,0,1,0,1,-1
2,563,233,167,77,228,555,928,869,452,360,1,0,0,0,0,-1
3,715,602,136,193,289,175,830,979,684,164,0,0,1,0,17,-1
4,672,519,136,231,292,122,869,971,625,45,0,0,1,0,17,-1


In [11]:
test.head()

,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil
0,489,355,197,237,333,456,948,909,460,404,1,0,0,0,3
1,626,333,257,192,337,398,971,881,393,165,0,0,1,0,16
2,657,103,242,137,252,236,857,802,464,636,0,1,0,0,8
3,543,175,182,232,365,500,905,846,456,317,1,0,0,0,0
4,746,888,121,150,261,197,783,916,680,274,0,0,1,0,16


In [12]:
val = train[10000:]
train = train[:10000]


In [13]:
y_train = train['label']
x_train = train.drop(['label'],axis=1)

y_val = val['label']
x_val = val.drop(['label'],axis=1)

In [14]:
x_val.head()

,Altitude,Azimuthal_angle,Incline,H_dist_Hydro,V_dist_Hydro,H_dist_Road,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,H_dist_Fire,Wilderness_Area_A,Wilderness_Area_B,Wilderness_Area_C,Wilderness_Area_D,Soil
10000,739,988,257,89,261,630,755,818,602,563,1,0,0,0,24
10001,564,183,272,192,303,766,924,791,362,193,1,0,0,0,0
10002,639,150,91,0,223,169,881,893,543,319,1,0,0,0,0
10003,665,863,303,68,257,98,637,869,779,343,0,1,0,0,18
10004,569,14,136,375,325,724,822,877,598,778,1,0,0,0,6


In [15]:
from sklearn.preprocessing import StandardScaler    # Using Standard Scalar to normalise our data
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.fit_transform(x_val)
test = sc.fit_transform(test)


/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shreyansh/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input

In [16]:
results = {}
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
model1= LogisticRegressionCV(cv=25, max_iter = 1000)
model1.fit(x_train,y_train)
test_res1=model1.predict(test)
results['LR']=model1.score(x_val,y_val)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
model2 =RandomForestClassifier (n_estimators=250, n_jobs=-1, max_features='auto')
model2.fit(x_train,y_train)
test_res2=model2.predict(test)
results['RF']=model2.score(x_val,y_val)

In [18]:
import xgboost
from xgboost import XGBClassifier
model3=XGBClassifier( n_estimators=250,max_depth=7,learning_rate=0.015,subsample=0.35)
model3.fit(x_train,y_train)
test_res3 = model3.predict(test)
results["XGB"]=model3.score(x_val,y_val)

In [19]:
from sklearn.ensemble import VotingClassifier
model7 = VotingClassifier(estimators=[('rf',model2),('lr',model1),('xgb',model3)] ,voting='soft', weights=[7,1,10])
model7.fit(x_train,y_train)
test_res4 = model7.predict(test)
results["VC"]=model7.score(x_val,y_val)

In [20]:
print(results)

{'LR': 0.8492693110647181, 'RF': 0.8705636743215032, 'XGB': 0.8718162839248434, 'VC': 0.872651356993737}


In [21]:

submission_df['Predicted'] = test_res3.tolist()

In [22]:
submission_df.to_csv('VotingClassifier.csv',index=False)